# Data Structures and Algorithms in Python - Exercises (Part 3)

- Link to student companion: https://bcs.wiley.com/he-bcs/Books?action=index&bcsId=8029&itemId=1118290275
- Some solutions to exercises: https://github.com/wdlcameron/Solutions-to-Data-Structures-and-Algorithms-in-Python
- More solutions (good ones!): https://github.com/ekeleshian/data_structures_and_algorithms/blob/master/object_oriented_programming.py

### P-2.36
Write a Python program to simulate an ecosystem containing two types of creatures, bears and ﬁsh.

The ecosystem consists of a river, which is modeled as a relatively large list.

Each element of the list should be a `Bear` object, a `Fish` object, or `None`.

In each time step, based on a random process, each animal either attempts to move into an adjacent list location or stay where it is.

- If two animals of the same type are about to collide in the same cell, then they stay where they are, but they create a new instance of that type of animal, which is placed in a random empty (i.e., previously `None`) location in the list.
- If a bear and a ﬁsh collide, however, then the ﬁsh dies (i.e., it disappears).

In [95]:
'''
river[target_position] = river[self_position] 
river[self_position] = None
'''        

l = list(['a', None])
print(l)
l[1] = l[0]
l[0] = None
print(l)

['a', None]
[None, 'a']


In [88]:
l[1] = l[0]

In [100]:
l[1]

'a'

In [6]:
# IMPORTS

import random
import functools


# LOGGER

import sys
from loguru import logger

logger.remove() # clear the decks
logger.add("debug.log", mode="w", backtrace=False) # specify mode="w" to overwrite each time, or a to append

logger.info("NEW RUN STARTED")

debug = False


# CONSTANTS

RIVER_LENGTH = 100
P_BEAR = 0.05
P_FISH = 0.1


def display_river(t):
    print(str(t).zfill(3), end=' ')
    for i, item in enumerate(river):
        if not item:
            print('-', end='')
        elif isinstance(item, Fish):
            print('f', end='') if item.get_is_born() and debug else print('F', end='')
        elif isinstance(item, Bear):
            print('\x1b[31mb\x1b[0m', end='') if item.get_is_born() and debug else print('\x1b[31mB\x1b[0m', end='')
    print('')

def find_free(l, pos):
    '''Returns index value of closest None value to a specified position in a list.
    Returns -1 if no None values are found'''
    
    offset, left_done, right_done, length = 0, False, False, len(l)
    
    while not (right_done and left_done):
        
        offset += 1
        
        pos_to_right = pos + offset
        pos_to_left = pos - offset
        
        if pos_to_right <= length-1:
            if not l[pos_to_right]:
                return pos_to_right
        else:
            right_done = True
        
        if pos_to_left >= 0:
            if not l[pos_to_left]:
                return pos_to_left
        else:
            left_done = True

    return -1

class Animal:
    def __init__(self, position, born = False):
        self._position = position
        self._processed = False
        self._born = born
        
    def get_processed(self):
        return self._processed
    
    def get_is_born(self):
        return self._born
    
    def set_processed(self):
        self._processed = True
    
    def reset_processed(self):
        self._processed = False

    def get_position(self):
        return self._position        

    def have_a_baby(self):
        """Creates a new instance of specified type in closest empty position to self.
        """
          
        empty_pos = find_free(river, self.get_position())
        
        if empty_pos == -1:
            global river_is_full
            river_is_full = True
            return
        
        if isinstance(self, Bear):
            if debug: logger.debug(f"Creating Bear with born=True at {empty_pos}")
            river[empty_pos] = Bear(empty_pos, born=True)
            river[empty_pos].set_processed()
            
        elif isinstance(self, Fish):
            if debug: logger.debug(f"Creating Fish with born=True at {empty_pos}")
            river[empty_pos] = Fish(empty_pos, born=True)
            river[empty_pos].set_processed()
        
    def __add__(self, other):
        """Operator override to allow straightforward addition of objects.
        Doesn't accept None as an argument"""
        
        self_position, other_position = self.get_position(), other.get_position()
        
        if type(self) == type(other):
            if debug: logger.debug(f"T{t}: Calling have_a_baby between {type(self), self_position} and {type(other), other_position}")
            self.have_a_baby()
            self.set_processed()
            other.set_processed()
            return "A new beast is born" # The creature is the same type

        elif isinstance(self, Bear): # bear eats fish
            
            if debug: logger.debug(f"T{t}: Bear at {self_position} eats fish at {other_position}")
            
            river[other_position] = river[self_position]
            
            river[self_position] = None
            river[other_position]._position = other_position
            river[other_position].set_processed()
                        
            return "There's been a murder" 

        elif isinstance(self, Fish): # fish moves in with bear, so disappears
            
            if debug: logger.debug(f"T{t}: Fish at {self_position} moves in with Bear, so is eaten")
            
            river[self_position] = None
            # self.set_processed() # Unecessary as we've removed ref to self from river
            
            return "Kamikaze"
            
    
    def move(self, direction):
        
        self_position = self.get_position()
        
        if debug: logger.debug(f"T{t}: Move from {self_position}, {direction}")
        
        if direction == 'upstream' and self_position == 0:
            return # Already at source. Can't move upstream
    
        elif direction == 'downstream' and self_position == len(river)-1:
            return # Already at mouth. Can't move downstream

        if direction == 'upstream':
            target_position = self_position - 1
        
        elif direction == 'downstream':
            target_position = self_position + 1

        if river[target_position]: # There's a creature in target location
            
            if debug: logger.debug(f"T{t}: Add {self_position} to {target_position}")
            
            river[self_position] + river[target_position]
        
        else: # There's a None in target location 
            
            river[target_position] = river[self_position]
            river[target_position]._position = target_position
            river[target_position].set_processed()
            river[self_position] = None
            
            
            if debug: logger.debug(f"T{t}: Move to {target_position}")
    
    def time_step(self):
        
        moves = bool(round(random.random())) # Does the creature move?
        
        if moves:
            up = bool(round(random.random())) # Does the creature move up or downstream?
            self.move('upstream') if up else self.move('downstream')
        
class Bear(Animal):
    def __init__(self, position, born = False):
        super().__init__(position, born = False)
        self._born = born

class Fish(Animal):
    def __init__(self, position, born = False):
        super().__init__(position, born = False)
        self._born = born

# Create empty river
river = [None] * RIVER_LENGTH
rivers = list()
river_is_full = False

log = []

# Populate river
for i in range(len(river)):
    r = random.choices(['bear', 'fish', None], [P_BEAR, P_FISH, 1-(P_BEAR + P_FISH)])[0]
    if r == 'bear':
        river[i] = Bear(i)
        #print("Creating Bear", i)
    elif r == 'fish':
        river[i] = Fish(i)
        #print("Creating Fish", i)
    else:
        pass

# Here we run our simulation

t = 0

def display_positions_labels():
    print("Tx ", "         ".join('0123456789'))

display_positions_labels()
display_river(t)

while not river_is_full:
    t += 1
    if t % 10 == 0 and debug: display_positions_labels()
    display_river(t) # t specifies the first character of the row
    for position in river: # iterate through all elements in river
        if position: # ... that aren't None
            #print(position)
            if not position.get_processed(): # ... that haven't been processed already
                    position.time_step()
            
    for position in river:
        if position:
            position.reset_processed()
            
    log.append(["----- COMPLETED TIME STEP -----"])      

display_river(t + 1)
print('River is full!')

Tx  0         1         2         3         4         5         6         7         8         9
000 --B--B-----------B-------F-----FF---------------------------F--------------------------F---F--B----F
001 --B--B-----------B-------F-----FF---------------------------F--------------------------F---F--B----F
002 ---BB------------B-------F----FFF---------------------------F--------------------------F---F-B----F-
003 --B-B------------B-------F---FF--F-------------------------F----------------------------F-F-B-----F-
004 --BB-------------B-------F--FFF---F-------------------------F---------------------------F--B------F-
005 -BBB------------B---------F-FFFF---F------------------------F---------------------------F--B-----F--
006 -BBB-------------B-------F-F-FF-F-F-------------------------F---------------------------F--B----F---
007 -BBBB-----------B--------F--FFFFFFFFF-----------------------F----------------------------FB-----F---
008 BBBBBBB---------B--------FFFFFFFFFFFFFF---------------------

__Follow up notes__

I spent a not-inconsiderable amount of time trying to work out why the following happens:
```
-F------
------F-
```

By capturing some rudimentary logs I identified the following

```
[['from: ', 23, 'downstream'],
 ['to: ', 24],
 ['from: ', 24, 'upstream'],
 ['to: ', 23],
 ['from: ', 33, 'downstream'],
 ['to: ', 34],
 ['from: ', 34, 'downstream'],
 ['to: ', 35],
 ['from: ', 35, 'downstream'],
 ['to: ', 36],
 ['from: ', 36, 'downstream'],
 ['to: ', 37],
 ['from: ', 37, 'downstream'],
 ['to: ', 38],
```

Here we can see that because our simulation iterates through each position in turn (`for i, position in enumerate(river)` - i.e. downstream), there are potential sequences of events that will cause multiple location changes while it would appear that only one should occur (33 -> 34, 34 -> 35 ... 37 -> 38!). Further, it's possible that a move downstream and then back upstream will occur within one timestep which looks like no movement in our representation (23 -> 24, 24 -> 23). The solution to this is to add in some kind of verification as to whether a particular object has already moved in the current time step, or to reorientate the simulation so that each object is iterated over, instead of each position in the river/list. Another soltuion would be to change the positions of each object after all positions had been processed.

One of the major hurdles in the project was the requirement to use a `list` as the representation of the river. This data structure is a bit limited as lists, by definition, support multiple identical items, so there's no method to identify each objects position without using some kind of tracking mechanism/property, and this got a little arduous and complicated. A `River` type/class, based on built-in `list` type/class would be the solution. This would also allow a neat representation which would improve on my `display_river()` method: simply calling the `River` object would cause a call to `__repr__()` which could return a sequence of `F`, `B`, or `-` as appropriate.

Also, working with `None` items within the list was a bit arduous too. A modified `None` type would allow operations such as `Bear + None` which would have made things more elegant: there are frequent checks (e.g. `if target:`) as to whether a position contains `None` to avoid problematic calls (e.g. `None.get_position()`).

I also stumbled when trying to keep a record of each timestep by appending `river` to a new list called `rivers` in the hope that I'd be able to view the state of river at differing timesteps by subscribing: `rivers[12]`. This doesn't work well at all, as `rivers.append(river)` results in aliases being established between e.g. `rivers[0]` and `river`, so when subsequently `river` changes, so does `rivers[0]`. Creating a `copy()` or a `deepcopy()` would help, but we'd need to make sure we can easily identify the individual objects, perhaps by exposing the position member data more readily.

Effectively, this is approaching something similar to a Turing machine!

I think I found another issue:

```
-F-B----
--FBB---
--FBBB--
```

```
['----- COMPLETED TIME STEP -----'],
 ['from: ', 12, 'downstream'], # fish
 ['to: ', 13],                 
 ['from: ', 15, 'downstream'], # bear
 ['to: ', 16],
 ['from: ', 16, 'upstream'],   # bear
 ['to: ', 15]
 ...
 ['----- COMPLETED TIME STEP -----'],
 ['from: ', 13, 'downstream'],
 ['from: ', 15, 'upstream'],
 ```
 
 In this instance, we're seeing a `Bear` spawn another without requiring another `Bear` to mate with! From exploring similar instances in subsequent runs, I established there were instances where objects of the same type were being added to themselves! This would happen if an object was moved, and then a subsequent comparison during the same time-step was processed on the moved object and on the position the object had moved from (in the case that the old position had not been set back to `None`).
 
 - Make sure to set the `river[self.get_position()]` to `None` following an operation which moves that object
 - Be aware of the difference of referring to `self` and to `river[self.get_position()]`
 
My method of creating a `processed` value for each item that has been manipulated already within a given timestep works quite well: rules like this make the system much more straightforward to debug.

The last observation I'd make is how important it is to reduce the complexity of the problem when troubleshooting: by changing the size of the river to just 10, and setting p(Bear) to 0 I was able to drill down (having implemented some debugging logs) and isolate strange occurences.

### P-2.37

Write a simulator, as in the previous project, but add a Boolean gender ﬁeld and a ﬂoating-point strength ﬁeld to each animal, using an Animal class as a base class.

If two animals of the same type try to collide, then they only create a new instance of that type of animal if they are of different genders. Otherwise, if two animals of the same type and gender try to collide, then only the one of larger strength survives.

In [ ]:
# IMPORTS

import random
import functools
import numpy as np

# LOGGER

import sys
from loguru import logger

logger.remove() # clear the decks
logger.add("debug.log", mode="w", backtrace=False) # specify mode="w" to overwrite each time, or a to append

logger.info("NEW RUN STARTED")

# CLASSES

class Strengths:
    """Generates a set of strengths for creatures in the river system
    Allows each strength value to be retrieved and hence assigned to each creature during
    initialisation of the system"""
    
    def __init__(self, size):
        
        """Generates a sampling distribution based on the Normal distribution"""
        count = 0
        strengths = [-1,2]
        while min(strengths) <= 0 or max(strengths) >= 1: # Ensures we only generate a distribution with values within acceptable range
            count += 1
            if count > 10: raise Exception("Too many attempts to build a normal distribution")
            strengths = np.random.normal(loc=0.5, scale=0.2,size=size)
        self._strengths = list(strengths) # So we can simply pop()
        
    def get_strength(self):
        """Pops a strength from our saved sampling distribution"""
        return self._strengths.pop()
    
    def __len__(self): # Just so we can create an iterable. Probs not useful
        return len(self._strengths)
    
    def __getitem__(self, val):  # Just so we can create an iterable. Probs not useful
        return self._strengths[val]

class Animal:
    def __init__(self, position, strength, born):
        self._is_female = choose_gender()
        self._strength = strength
        self._position = position
        self._born = born
        self._processed = False
        
    def get_processed(self):
        return self._processed
    
    def get_is_born(self):
        return self._born
    
    def set_processed(self):
        self._processed = True
    
    def reset_processed(self):
        self._processed = False

    def get_position(self):
        return self._position
    
    def _set_position(self, val):
        """Brutish method to set the position of a creature. Not for general consumption!"""
        self._position = val
    
    def get_is_female(self):
        return self._is_female
    
    def get_strength(self):
        return self._strength

    def have_a_baby(self, other):
        """Creates a new instance of specified type in closest empty position to self.
        Relies on function find_free()
        Relies on Animal class member methods:
        get_position(), get_strength(), set_processed()
        """
          
        empty_pos = find_free(river, self.get_position())
        
        if empty_pos == -1:
            global halt_simulation
            halt_simulation = "River is full!"
            return
        
        new_strength = np.mean([self.get_strength(), other.get_strength()])
        
        if isinstance(self, Bear):
            if debug: logger.debug(f"Creating Bear with born=True at {empty_pos}")
            river[empty_pos] = Bear(empty_pos, strength = new_strength, born=True)
            river[empty_pos].set_processed()
            
        elif isinstance(self, Fish):
            if debug: logger.debug(f"Creating Fish with born=True, strength={new_strength}, at {empty_pos}")
            river[empty_pos] = Fish(empty_pos, strength = new_strength, born=True)
            river[empty_pos].set_processed()

        
    def __add__(self, other):
        """Operator override to allow straightforward addition of objects.
        Doesn't accept None as an argument. Reliant on Animal class member methods:
        get_position(), get_is_female(), have_a_baby(), set_processed()"""
        
        self_position, other_position = self.get_position(), other.get_position()
        
        if type(self) == type(other): # Animals are the same type
            
            if not self.get_is_female() and other.get_is_female(): # not has higher priority than boolean operation and
            
                if debug: logger.debug(f"T{t}: Calling have_a_baby between {type(self), self_position} and {type(other), other_position}")
                self.have_a_baby(other)
                self.set_processed()
                other.set_processed()
                return "A new beast is born" # The creature is the same type
            else:
                # Different gender beasts are trying to enter the same position
                # self_strength, other_strength = self.get_strength(), other.get_strength()
                # if self_strength >= 
                
                pass

        elif isinstance(self, Bear): # bear eats fish
            #river.insert(other_position, river.pop(self_position))
            
            if debug: logger.debug(f"T{t}: Bear at {self_position} eats fish at {other_position}")
            
            river[other_position] = river[self_position]
            
            river[self_position] = None
            river[other_position]._set_position(other_position)
            river[other_position].set_processed()
                        
            return "There's been a murder" 

        elif isinstance(self, Fish): # fish moves in with bear, so disappears
            
            if debug: logger.debug(f"T{t}: Fish at {self_position} moves in with Bear, so is eaten")
            
            river[self_position] = None
            # self.set_processed() # Unecessary as we've removed ref to self from river
            
            return "Kamikaze"
            
    def move(self, direction):
        """Reliant on Animal class member methods:
        _set_position(), get_position(), .set_processed()"""
        
        self_position = self.get_position()
        
        if debug: logger.debug(f"T{t}: Move from {self_position}, {direction}")
        
        if direction == 'upstream' and self_position == 0:
            return # Already at source. Can't move upstream
    
        elif direction == 'downstream' and self_position == len(river)-1:
            return # Already at mouth. Can't move downstream

        if direction == 'upstream':
            target_position = self_position - 1
        
        elif direction == 'downstream':
            target_position = self_position + 1

        if river[target_position]: # There's a creature in target location
            
            if debug: logger.debug(f"T{t}: Add {self_position} to {target_position}")
            
            river[self_position] + river[target_position]
        
        else: # There's a None in target location 

            # river.insert(target_position, river.pop(self_position))
            
            river[target_position] = river[self_position]
            river[target_position]._set_position(target_position)
            river[target_position].set_processed()
            river[self_position] = None
            
            
            if debug: logger.debug(f"T{t}: Move to {target_position}")
    
    def time_step(self):
        """Reliant on Animal calss member methods:
        move()"""
        
        #print("time_step(self)",self)
        moves = bool(round(random.random())) # Does the creature move?
        
        if moves:
            up = bool(round(random.random())) # Does the creature move up or downstream?
            self.move('upstream') if up else self.move('downstream')

    def __repr__(self):
        
        # elif isinstance(item, Fish):
        #     
        # elif isinstance(item, Bear):
        #     print('\x1b[31mb\x1b[0m', end='') if item.get_is_born() and debug else print('\x1b[31mB\x1b[0m', end='')
        
        gender_symbol = GENDER_SYMBOL_FEMALE if self.get_is_female() else GENDER_SYMBOL_MALE
        gender_symbol = GENDER_SYMBOL_FEMALE if self.get_is_female() else GENDER_SYMBOL_MALE
        
        if isinstance(self, Bear):
            if self.get_is_born() and debug:
                return 'b' + gender_symbol
            else:
                return 'B' + gender_symbol
            
        elif isinstance(self, Fish):
            if self.get_is_born() and debug:
                return 'f' + gender_symbol
            else:
                return 'F' + gender_symbol
            
    def __len__(self):
        return len(self.__repr__())
            
class Bear(Animal):
    def __init__(self, position, strength, born = False):
        super().__init__(position, strength, born)
        

class Fish(Animal):
    def __init__(self, position, strength, born = False):
        super().__init__(position, strength, born)

def choose_gender():
    return bool(np.random.binomial(n=1, p=P_FEMALE, size=int(1)))


# FUNCTIONS

# def show_strength(text,strength_val):
#     colour_key = int(round(strength_val, 1)*10)
#     return "\x1b[38;5;{}m{}\x1b[0m".format(colour_range[colour_key], text)

def show_strength(text, strength_val, transform = False):
    # print('colour_key', colour_key)
    if transform:
        s = gaussian(GAUSSIAN_A, GAUSSIAN_B, GAUSSIAN_C, strength_val)
        s = 0.1 if s < 0.1 else s
    colour_key = int((round(s, 1))*NUM_SHADES) - 1
    return "\x1b[38;5;{}m{}\x1b[0m".format(colour_range[colour_key], text)

def display_river(t):
    print(str(t).zfill(3), end=' ')
    for i, item in enumerate(river):
        if not item:
            print('--', end='')
        else:
            s = item.get_strength()
            print(show_strength(item, s, transform = True), end='')
        print(' ', end='')
    print('')

def find_free(l, pos):
    '''Returns index value of closest None value to a specified position in a list.
    Returns -1 if no None values are found'''
    
    offset, left_done, right_done, length = 0, False, False, len(l)
    
    while not (right_done and left_done):
        
        offset += 1
        
        pos_to_right = pos + offset
        pos_to_left = pos - offset
        
        if pos_to_right <= length-1:
            if not l[pos_to_right]:
                return pos_to_right
        else:
            right_done = True
        
        if pos_to_left >= 0:
            if not l[pos_to_left]:
                return pos_to_left
        else:
            left_done = True

    return -1

def gaussian(a, b, c, x):
    """Gaussian transformation function
    a: height of the curve's peak
    b: centre
    c: standard deviation
    x: integer"""
    
    power = (-(x-b)**2)/(2*c**2)
    return a * np.exp(power)

# MAIN PROGRAM

# CONSTANTS

RIVER_LENGTH = 50
DEFAULT_IS_FEMALE = False
P_BEAR = 0.2
P_FISH = 0.2
P_FEMALE = 0.7
GENDER_SYMBOL_FEMALE = '\u2640'
GENDER_SYMBOL_MALE = '\u2642'

# colouring/shading
GAUSSIAN_A = 1   # height of the curve's peak
GAUSSIAN_B = 1 # centre
GAUSSIAN_C = 0.50 # standard deviation

COLOUR_BRACKET_MIN = 235 # 196 Red
COLOUR_BRACKET_MAX = 255 # 201 Pink
NUM_SHADES = 10

# Initialise colouring of StdOut

step_size = int((COLOUR_BRACKET_MAX-COLOUR_BRACKET_MIN)/NUM_SHADES)
colour_range = range(COLOUR_BRACKET_MAX, COLOUR_BRACKET_MIN, -step_size)

# Create empty river
river = [None] * RIVER_LENGTH
rivers = list()
halt_simulation = False

log = []

# Populate river
strengths = Strengths(RIVER_LENGTH)

for i in range(len(river)):
    r = random.choices(['bear', 'fish', None], [P_BEAR, P_FISH, 1-(P_BEAR + P_FISH)])[0]
    if r == 'bear':
        river[i] = Bear(i, strength = strengths.get_strength())
    elif r == 'fish':
        river[i] = Fish(i, strength = strengths.get_strength())
    else:
        river[i] = None

# Here we run our simulation

debug = True

t = 0

def display_positions_labels():
    length_floor_10 = RIVER_LENGTH // 10
    labels = "".join([str(x)+'  ' for x in range(length_floor_10)])
    print("Tx ", (9 * " ").join(labels))

    
# print example of shading
for i in range(10):
    i = i/10
    print(show_strength('B', i, transform = True), end=' ')
print()

display_positions_labels()
display_river(t)

while not halt_simulation:
    t += 1
    if t % 10 == 0 and debug: display_positions_labels()
    display_river(t) # t specifies the first character of the row
    for position in river: # iterate through all elements in river
        if position: # ... that aren't None
            #print(position)
            if not position.get_processed(): # ... that haven't been processed already
                    position.time_step()
    
    for position in river:
        if position:
            position.reset_processed()
    
    log.append(["----- COMPLETED TIME STEP -----"])      
    
display_river(t + 1)
print('Simulation halted because:', halt_simulation)

In [2]:
######################################################
######################################################
###
### And now implement strengths
###
######################################################
######################################################

# IMPORTS

import random
import functools
import numpy as np

# LOGGER

import sys
from loguru import logger

logger.remove() # clear the decks
logger.add("debug.log", mode="w", backtrace=False) # specify mode="w" to overwrite each time, or a to append

logger.info("NEW RUN STARTED")

# CLASSES

class Strengths:
    """Generates a set of strengths for creatures in the river system
    Allows each strength value to be retrieved and hence assigned to each creature during
    initialisation of the system"""
    
    def __init__(self, size):
        
        """Generates a sampling distribution based on the Normal distribution"""
        count = 0
        strengths = [-1,2]
        while min(strengths) <= 0 or max(strengths) >= 1: # Ensures we only generate a distribution with values within acceptable range
            count += 1
            if count > 10: raise Exception("Too many attempts to build a normal distribution")
            strengths = np.random.normal(loc=0.5, scale=0.2,size=size)
        self._strengths = list(strengths) # So we can simply pop()
        
    def get_strength(self):
        """Pops a strength from our saved sampling distribution"""
        return self._strengths.pop()
    
    def __len__(self): # Just so we can create an iterable. Probs not useful
        return len(self._strengths)
    
    def __getitem__(self, val):  # Just so we can create an iterable. Probs not useful
        return self._strengths[val]

class Animal:
    def __init__(self, position, strength, born):
        self._is_female = choose_gender()
        self._strength = strength
        self._position = position
        self._born = born
        self._processed = False
        
    def get_processed(self):
        return self._processed
    
    def get_is_born(self):
        return self._born
    
    def set_processed(self):
        self._processed = True
    
    def reset_processed(self):
        self._processed = False

    def get_position(self):
        return self._position
    
    def _set_position(self, val):
        """Brutish method to set the position of a creature. Not for general consumption!"""
        self._position = val
        self.set_processed()
    
    def get_is_female(self):
        return self._is_female
    
    def get_strength(self):
        return self._strength

    def have_a_baby(self, other):
        """Creates a new instance of specified type in closest empty position to self.
        Strength of new instance to be average of its parents.
        Relies on function find_free()
        Relies on Animal class member methods:
        get_position(), get_strength(), set_processed()
        """
          
        empty_pos = find_free(river, self.get_position())
        
        if empty_pos == -1:
            global halt_simulation
            halt_simulation = "River is full!"
            return
        
        new_strength = np.mean([self.get_strength(), other.get_strength()])
        
        if isinstance(self, Bear):
            if debug: logger.debug(f"Creating Bear with born=True at {empty_pos}")
            river[empty_pos] = Bear(empty_pos, strength = new_strength, born=True)
            river[empty_pos].set_processed()
            
        elif isinstance(self, Fish):
            if debug: logger.debug(f"Creating Fish with born=True, strength={new_strength}, at {empty_pos}")
            river[empty_pos] = Fish(empty_pos, strength = new_strength, born=True)
            river[empty_pos].set_processed()

        
    def __add__(self, other):
        """Operator override to allow straightforward addition of objects.
        Doesn't accept None as an argument. Reliant on Animal class member methods:
        get_position(), get_is_female(), have_a_baby(), set_processed()"""
        
        self_position, other_position = self.get_position(), other.get_position()
        
        if type(self) == type(other): # Animals are the same type
            
            if not self.get_is_female() and other.get_is_female(): # not has higher priority than boolean operation and
            
                if debug: logger.debug(f"T{t}: Calling have_a_baby between {type(self), self_position} and {type(other), other_position}")
                self.have_a_baby(other)
                self.set_processed()
                other.set_processed()
                return "A new beast is born" # The creature is the same type
            else:
                # Beasts of the same type and gender are trying to move in to the same spot.
                # If self is stronger, it moves to other's position and its original pos is set to None
                # If other is stronger, the reverse happens.
                # If they are of equal strength thenthere's no change.
                
                self_strength, other_strength = self.get_strength(), other.get_strength()
                
                if self_strength > other_strength:
                    
                    river[other_position] = river[self_position]; river[self_position] = None
                    river[other_position]._set_position(other_position); river[other_position].set_processed()
                    
                elif other_strength > self_strength:
                    
                    river[self_position] = river[other_position]; river[other_position] = None
                    river[self_position]._set_position(self_position); river[self_position].set_processed()
                
                else:
                    pass

        elif isinstance(self, Bear): # bear eats fish
            #river.insert(other_position, river.pop(self_position))
            
            if debug: logger.debug(f"T{t}: Bear at {self_position} eats fish at {other_position}")
            
            river[other_position] = river[self_position]; river[self_position] = None
            river[other_position]._set_position(other_position); river[other_position].set_processed()
                        
            return "There's been a murder" 

        elif isinstance(self, Fish): # fish moves in with bear, so disappears
            
            if debug: logger.debug(f"T{t}: Fish at {self_position} moves in with Bear, so is eaten")
            
            river[self_position] = None
            # self.set_processed() # Unecessary as we've removed ref to self from river
            
            return "Kamikaze"
            
    def move(self, direction):
        """Reliant on Animal class member methods:
        _set_position(), get_position(), .set_processed()"""
        
        self_position = self.get_position()
        
        if debug: logger.debug(f"T{t}: Move from {self_position}, {direction}")
        
        if direction == 'upstream' and self_position == 0:
            return # Already at source. Can't move upstream
    
        elif direction == 'downstream' and self_position == len(river)-1:
            return # Already at mouth. Can't move downstream

        if direction == 'upstream':
            target_position = self_position - 1
        
        elif direction == 'downstream':
            target_position = self_position + 1

        if river[target_position]: # There's a creature in target location
            
            if debug: logger.debug(f"T{t}: Add {self_position} to {target_position}")
            
            river[self_position] + river[target_position]
        
        else: # There's a None in target location 

            #river[target_position] = river[self_position]
            #river[self_position] = None            
            
            # Use simultaneous assignment to achieve the same effect as above couple of lines
            river[target_position], river[self_position] = river[self_position], river[target_position]
            
            river[target_position]._set_position(target_position)

            if debug: logger.debug(f"T{t}: Move to {target_position}")
    
    def time_step(self):
        """Reliant on Animal class member methods:
        move()"""
        
        #print("time_step(self)",self)
        moves = bool(round(random.random())) # Does the creature move?
        
        if moves:
            up = bool(round(random.random())) # Does the creature move up or downstream?
            self.move('upstream') if up else self.move('downstream')

    def __repr__(self):
        
        # elif isinstance(item, Fish):
        #     
        # elif isinstance(item, Bear):
        #     print('\x1b[31mb\x1b[0m', end='') if item.get_is_born() and debug else print('\x1b[31mB\x1b[0m', end='')
        
        gender_symbol = GENDER_SYMBOL_FEMALE if self.get_is_female() else GENDER_SYMBOL_MALE
        
        if isinstance(self, Bear):
            if self.get_is_born() and debug:
                return 'b' + gender_symbol
            else:
                return 'B' + gender_symbol
            
        elif isinstance(self, Fish):
            if self.get_is_born() and debug:
                return 'f' + gender_symbol
            else:
                return 'F' + gender_symbol
            
    def __len__(self):
        return len(self.__repr__())
            
class Bear(Animal):
    def __init__(self, position, strength, born = False):
        super().__init__(position, strength, born)
        

class Fish(Animal):
    def __init__(self, position, strength, born = False):
        super().__init__(position, strength, born)

def choose_gender():
    return bool(np.random.binomial(n=1, p=P_FEMALE, size=int(1)))


# FUNCTIONS

# def show_strength(text,strength_val):
#     colour_key = int(round(strength_val, 1)*10)
#     return "\x1b[38;5;{}m{}\x1b[0m".format(colour_range[colour_key], text)

def show_strength(text, strength_val, transform = False):
    # print('colour_key', colour_key)
    if transform:
        s = gaussian(GAUSSIAN_A, GAUSSIAN_B, GAUSSIAN_C, strength_val)
        s = 0.1 if s < 0.1 else s
    colour_key = int((round(s, 1))*NUM_SHADES) - 1
    return "\x1b[38;5;{}m{}\x1b[0m".format(colour_range[colour_key], text)

def display_river(t):
    print(str(t).zfill(3), end=' ')
    for i, item in enumerate(river):
        if not item:
            print('--', end='')
        else:
            s = item.get_strength()
            print(show_strength(item, s, transform = True), end='')
        print(' ', end='')
    print('')

def find_free(l, pos):
    '''Returns index value of closest None value to a specified position in a list.
    Returns -1 if no None values are found'''
    
    offset, left_done, right_done, length = 0, False, False, len(l)
    
    while not (right_done and left_done):
        
        offset += 1
        
        pos_to_right = pos + offset
        pos_to_left = pos - offset
        
        if pos_to_right <= length-1:
            if not l[pos_to_right]:
                return pos_to_right
        else:
            right_done = True
        
        if pos_to_left >= 0:
            if not l[pos_to_left]:
                return pos_to_left
        else:
            left_done = True

    return -1

def gaussian(a, b, c, x):
    """Gaussian transformation function
    a: height of the curve's peak
    b: centre
    c: standard deviation
    x: integer"""
    
    power = (-(x-b)**2)/(2*c**2)
    return a * np.exp(power)

# MAIN PROGRAM

# CONSTANTS

RIVER_LENGTH = 20
DEFAULT_IS_FEMALE = False
P_BEAR = 0.4
P_FISH = 0.4
P_FEMALE = 0.7
GENDER_SYMBOL_FEMALE = '\u2640'
GENDER_SYMBOL_MALE = '\u2642'

# colouring/shading
GAUSSIAN_A = 1   # height of the curve's peak
GAUSSIAN_B = 1 # centre
GAUSSIAN_C = 0.50 # standard deviation

COLOUR_BRACKET_MIN = 235 # 196 Red
COLOUR_BRACKET_MAX = 255 # 201 Pink
NUM_SHADES = 10

# Initialise colouring of StdOut

step_size = int((COLOUR_BRACKET_MAX-COLOUR_BRACKET_MIN)/NUM_SHADES)
colour_range = range(COLOUR_BRACKET_MAX, COLOUR_BRACKET_MIN, -step_size)

# Create empty river
river = [None] * RIVER_LENGTH
rivers = list()
halt_simulation = False

log = []

# Populate river
strengths = Strengths(RIVER_LENGTH)

for i in range(len(river)):
    r = random.choices(['bear', 'fish', None], [P_BEAR, P_FISH, 1-(P_BEAR + P_FISH)])[0]
    if r == 'bear':
        river[i] = Bear(i, strength = strengths.get_strength())
    elif r == 'fish':
        river[i] = Fish(i, strength = strengths.get_strength())
    else:
        river[i] = None

# Here we run our simulation

debug = True

t = 0

def display_positions_labels():
    length_floor_10 = RIVER_LENGTH // 10
    labels = "".join([str(x)+'  ' for x in range(length_floor_10)])
    print("Tx ", (9 * " ").join(labels))

    
# print example of shading
for i in range(10):
    i = i/10
    print(show_strength('B', i, transform = True), end=' ')
print()

display_positions_labels()
display_river(t)

while not halt_simulation:
    t += 1
    if t % 10 == 0 and debug: display_positions_labels()
    if t == 100: halt_simulation = 't = 100'
    display_river(t) # t specifies the first character of the row
    for position in river: # iterate through all elements in river
        if position: # ... that aren't None
            #print(position)
            if not position.get_processed(): # ... that haven't been processed already
                    position.time_step()
    
    for position in river:
        if position:
            position.reset_processed()
    
    if debug: log.append(["----- COMPLETED TIME STEP -----"])      
    
display_river(t + 1)
print('Simulation halted because:', halt_simulation)

B B B B B B B B B B 
Tx  0                             1                    
000 F♀ -- B♂ B♀ B♀ B♂ B♀ F♀ B♂ -- B♀ -- B♀ -- F♀ -- -- -- F♀ B♂ 
001 F♀ -- B♂ B♀ B♀ B♂ B♀ F♀ B♂ -- B♀ -- B♀ -- F♀ -- -- -- F♀ B♂ 
002 F♀ -- -- B♂ -- B♀ B♀ -- B♂ -- B♀ B♀ -- -- F♀ -- -- -- F♀ B♂ 
003 F♀ -- -- B♂ -- B♀ -- B♀ B♂ -- b♀ -- B♀ F♀ -- -- -- -- F♀ B♂ 
004 F♀ -- B♂ -- -- -- B♀ B♀ B♂ -- b♀ -- -- B♀ -- -- -- -- B♂ -- 
005 F♀ -- -- B♂ -- -- B♀ -- B♀ -- b♀ -- -- -- B♀ -- -- B♂ -- -- 
006 F♀ -- B♂ -- -- -- B♀ B♀ -- -- -- b♀ -- -- -- B♀ B♂ -- -- -- 
007 F♀ -- B♂ -- -- -- B♀ -- -- -- -- b♀ -- -- -- B♀ -- B♂ -- -- 
008 -- F♀ -- B♂ -- -- -- B♀ -- -- b♀ -- -- -- -- B♀ B♂ -- -- -- 
009 F♀ -- -- B♂ -- -- -- B♀ -- b♀ -- -- -- -- -- B♀ -- B♂ -- -- 
Tx  0                             1                    
010 -- F♀ -- -- B♂ -- -- B♀ -- -- b♀ -- -- -- -- -- B♀ B♂ -- -- 
011 F♀ -- -- B♂ -- -- B♀ -- -- -- b♀ -- -- -- -- -- B♀ B♂ b♀ -- 
012 F♀ -- -- B♂ -- B♀ -- -- -- -- -- b♀ -- -- -- -- -- B♀ b♀ -- 
013 -- F♀ -- B♂ -- -- 

In [ ]:
# unique_ids = []

# def get_unique_id():
    # """Issues and records unique IDs.
    # Doesn't work very well because the comparison r not in doesn't take into account the zfill operation"""
#     r = ''
#     while r not in unique_ids:
#         r = str(round(random.random()*1000)).zfill(3)
#         unique_ids.append(r)
#     return r

In [ ]:
#     def move_upstream(self):
        
#         position = self.get_position()
        
#         if position == 0: # Already at source. Can't move upstream
#             return
        
#         else:
#             # move upstream
            
#             if river[position-1]: # There's a creature in target location
#                 # Alter data type to include operator override and then we can simply add positions here and the member functions will take care of logic
#                 if river[position-1].get_type() == self.get_type(): # The creature is the same type
#                     self.have_a_baby()
                    
#                 elif (river[position-1].get_type() == 'bear') and (self.get_type() == 'fish'):
#                     self.die()

#                 elif (river[position-1].get_type() == 'fish') and (self.get_type() == 'bear'):  # Defining __add__ & __radd__ will remove this step
#                     river[position-1].die()

#             else:
#                 self._position -= 1
#                 river[position-1] = river[position]
#                 river[position] = None
        

#     def move_downstream(self):
        
#         position = self.get_position()
        
#         if position == len(river)-1: # Already at mouth. Can't move downstream
#             pass
        
#         else:
#             # move downstream
#             if river[position+1]:
#                 if river[position+1].get_type() == self.get_type():
#                     self.have_a_baby()
                    
#                 elif (river[position+1].get_type() == 'bear') and (self.get_type() == 'fish'):
#                     self.die()

#                 elif (river[position+1].get_type() == 'fish') and (self.get_type() == 'bear'):
#                     river[position+1].die()

#             else:
#                 self._position += 1
#                 river[position+1] = river[position]
#                 river[position] = None

In [ ]:
# Before merging the below:

#         if direction == 'upstream':

#             if self_position == 0: return # Already at source. Can't move upstream

#             target = river[self_position-1]
            
#             if target: self + target # There's a creature in target location

#             else: self.set_position(self_position - 1) # There's a None in target location 

#         elif direction == 'downstream':

#             if self_position == len(river)-1: return # Already at mouth. Can't move downstream
        
#             target = river[self_position+1]

#             if target: self + target

#             else: self.set_position(self_position + 1)

### P-2.39

Develop an inheritance hierarchy based upon a `Polygon` class that has abstract methods `area()` and `perimeter()`. Implement classes `Triangle`, `Quadrilateral`, `Pentagon`, `Hexagon`, and `Octagon` that extend this base class, with the obvious meanings for the `area()` and `perimeter()` methods. Also implement classes, `IsoscelesTriangle`, `EquilateralTriangle`, `Rectangle`, and `Square`, that have the appropriate inheritance relationships. Finally, write a simple program that allows users to create polygons of the various types and input their geometric dimensions, and the program then outputs their area and perimeter.

For extra effort, allow users to input polygons by specifying their vertex coordinates and be able to test if two such polygons are similar.

In [308]:
from abc import ABC, abstractmethod

class Polygon(ABC): # Polygon is derived from ABC, which is derived from object()
    """Abstract Base Class
    Provides methods area and perimeter for concrete subclassess"""
    
    @abstractmethod # Indicates that this method must be overriden by derived object
    def get_area(self):
        pass

    @abstractmethod
    def get_perimeter(self):
        pass
    
    def __init__(self, len_sides=[], verts=[]):

        self._len_sides = len_sides
        self._num_sides = len(len_sides)
        self._num_vertices = len(verts)
    
    def get_len_sides(self):
        return self._len_sides
    
    def get_num_sides(self):
        return self._num_sides
    
    def get_num_vertices(self):
        return self._num_vertices

class Triangle(Polygon): # Triangle is derived from Polygon. Assumed we have measurements of all sides
    
    def __init__(self, len_sides, verts=[]):
        super().__init__(len_sides, verts)
        
        if len(len_sides) != 3: # No input validation
            raise ValueError("need length of exactly three sides")
    
    def get_area(self):
        a, b, c = self.get_len_sides()
        s = (a + b + c)/2 # semi-perimeter
        area = (s*(s-a)*(s-b)*(s-c))**0.5 # Heron's forumla
        return area
    
    def get_perimeter(self):
        a, b, c = self.get_len_sides()
        return a+b+c

class IsocelesTriangle(Polygon): # IsocelesTriangle is derived from Polygon
    
    def __init__(self, a, b):
        super().__init__([a,a,b], verts=[])
        self._len_a, self._len_b = a, b
    
    def get_area(self):
        a, b = self._len_a, self._len_b
        return (b*((a**2) - ((b/2)**2)) ** 0.5) / 2
    
    def get_perimeter(self):
        return 2 * self._len_a + self._len_b
    
class EquilateralTriangle(Polygon): # EquilateralTriangle is derived from Polygon
    
    def __init__(self, a):
        super().__init__([a,a,a], verts=[])
        self._len_a = a
    
    def get_area(self):
        a = self._len_a
        return ((3**0.5)/4) * (a**2)
    
    def get_perimeter(self):
        a = self._len_a
        return 3 * a
        
class Quadrilateral(Polygon):
    
    def __init__(self, len_sides, verts=[]):
        
        if len(len_sides) != 4:
            raise ValueError("Exactly 4 side lengths required")
        super().__init__(len_sides, verts)
    
    def get_area(self):
        return "area of quadrilateral"
    
    def get_perimeter(self):
        p = 0
        for side in self.get_len_sides():
            p = p + side
        return p
    
class Rectangle(Quadrilateral):
    
    def __init__(self, len_sides, verts=[]):
        
        if len(len_sides) != 2:
            raise ValueError("Exactly 2 side lengths required")
        
        super().__init__([len_sides[0],len_sides[1],len_sides[0],len_sides[1]], verts)
    
    def get_area(self):
        a = self.get_len_sides()[0]
        b = self.get_len_sides()[1]
        return a * b
    
    def get_perimeter(self):
        a = self.get_len_sides()[0]
        b = self.get_len_sides()[1]
        return 2 * a + 2 * b
    
class Square(Rectangle): # A square is a special rectangle
    
    def __init__(self, len_sides, verts=[]):
        
        if len(len_sides) != 1:
            raise ValueError("Exactly 1 side lengths required")

        super().__init__([len_sides[0], len_sides[0]], verts)
    
    def get_area(self):
        a = self.get_len_sides()[0]
        return a ** 2
    
    def get_perimeter(self):
        a = self.get_len_sides()[0]
        return 4 * a

In [301]:
tri = Triangle([3,4,5])
tri.get_len_sides(), tri.get_num_sides(), tri.get_num_vertices(), tri.get_area(), tri.get_perimeter()

([3, 4, 5], 3, 0, 6.0, 12)

In [302]:
isotri = IsocelesTriangle(2,3)
isotri.get_len_sides(), isotri.get_num_sides(), isotri.get_num_vertices(), isotri.get_area(), isotri.get_perimeter()

([2, 2, 3], 3, 0, 1.984313483298443, 7)

In [303]:
equitri = EquilateralTriangle(2)
equitri.get_len_sides(), equitri.get_num_sides(), equitri.get_num_vertices(), equitri.get_area(), equitri.get_perimeter()

([2, 2, 2], 3, 0, 1.7320508075688772, 6)

In [309]:
quad = Quadrilateral([2,3,4,5])
quad.get_len_sides(), quad.get_num_sides(), quad.get_num_vertices(), quad.get_area(), quad.get_perimeter()

([2, 3, 4, 5], 4, 0, 'area of quadrilateral', 14)

In [311]:
rectangle = Rectangle([2,3])
rectangle.get_len_sides(), rectangle.get_num_sides(), rectangle.get_num_vertices(), rectangle.get_area(), rectangle.get_perimeter()

([2, 3, 2, 3], 4, 0, 6, 10)

In [310]:
square = Square([2])
square.get_len_sides(), square.get_num_sides(), square.get_num_vertices(), square.get_area(), square.get_perimeter()

([2, 2, 2, 2], 4, 0, 4, 8)

I found some guidance at the following link: https://www.sitepoint.com/community/t/typeerror-new-missing-2-required-positional-arguments-bases-and-namespace/316646

I wasn't really sure how far to go with this. Looking at the above link helped me get a bit of a better perspective. For example, I'm not quite sure why there needs to be different classes for Isoceles/Equilateral triangles... the same method (Heron's method) canbe used to calculate area. The inheritance structure would be better I feel if there was a requirements also to create Right Angle and Scalene triangles: that would have all types of triangles covered.

I haven't added the pentagon or hexagon subclasses above. They should be derived from the `Polygon` class. The challenge in implementing them is simply to ensure that the formula for the `get_area()` member method is designed correctly, which is more of a maths problem than a coding problem.

Also, I haven't progressed the following bit:

> For extra effort, allow users to input polygons by specifying their vertex coordinates and be able to test if two such polygons are similar.

This could be approached in a few ways, depending on how we interpret the term "similar":

1. By considering only the area and perimeter values of each. We could simply compare these values. If the values are the same, we can state that the polygons might be similar. We can't be sure though.
2. By considering the length of sides of each polygon. If the polygon sides are the same length, and in the same order, then we can state that the polygons are identical, regardless of orientation.
3. By considering the length of sides of each polygon in proportion to each other. We can state that polygons are similar if we consider a scaling factor.

I feel like this is taking me down a bit of a rabbit hole and I'm going to move on. I could spend time researching how to compare the geometry of objects, but for now I think better to move to the next topic.